In [1]:
import sys
sys.path.append('../../network')
import network
import pandas as pd
import copy

colors = {'p__Firmicutes': '#85D696', 
          'p__Bacteroidetes': '#7EC9CE', 
          'p__Proteobacteria': '#FFC68A', 
          'p__Actinobacteria': '#999FDB', 
          'p__Spirochaetes':'#C0E481', 
          'p__Euryarchaeota': '#CC99DB', 
          'other': '#AAAAAA'}

bg_colors = {'#85D696': '#D1F0D8', 
            '#7EC9CE': '#C8E8EA', 
            '#FFC68A': '#FFDFBD', 
            '#999FDB': '#BFC2E8', 
            '#C0E481':'#D5EDAB', 
            '#CC99DB': '#EBA8FF', 
            '#AAAAAA': '#B3B3B3'}


colors_reverse = {}
phy_table = pd.read_csv('../../data/cMD.select_2008.species_phylum.tsv', sep='\t', header=None, index_col=None)
colors_dict = {}
phy_dict = {}
for phy, color in colors.items():
    colors_dict[color] = 0
    colors_reverse[color] = phy
    phy_dict[phy] = []
total_num = phy_table.shape[0]

for i in range(total_num):
    sp = phy_table.iloc[i, 0]
    phy = phy_table.iloc[i, 1]
    #print(phy, sp)
    if phy not in colors.keys():
        phy = 'other'
    colors_dict[colors[phy]] += 1
    phy_dict[phy].append(sp)

rnode = 5
width, origin, polar_df = network.assign_pos(total_num, rnode, margin=0)


In [2]:

def complete_df(layout, phy_dict, colors_reverse, bg_color_dict):
    col_order = ['id', 'theta', 'r', 'color', 'phy', 'bg_color']
    for idx in layout.index:
        color = layout.loc[idx, 'color'] 
        phy = colors_reverse[color]
        #print(color, phy)
        layout.loc[idx, 'id'] = phy_dict[phy].pop()
        layout.loc[idx, 'phy'] = phy
        layout.loc[idx, 'bg_color'] = bg_color_dict[color]
    return layout[col_order]


In [3]:
# test ring_layout
layout = network.ring_layout(colors_dict, polar_df)
complete_df(layout, copy.deepcopy(phy_dict), colors_reverse, bg_colors).to_csv("ring_sp_layout.tsv", sep='\t', index=None)

# test sector_layout
layout = network.sector_layout(colors_dict, polar_df)
complete_df(layout, copy.deepcopy(phy_dict), colors_reverse, bg_colors).to_csv("sector_sp_layout.tsv", sep='\t', index=None)

# test scale_layout
layout, seeds = network.scale_layout(colors_dict, polar_df, width)
complete_df(layout, copy.deepcopy(phy_dict), colors_reverse, bg_colors).to_csv("scale_sp_layout.tsv", sep='\t', index=None)

with open('config.txt', 'w') as fp:
    fp.write("width = {}\norigin = {},{}\nr = {}".format(width, origin[0], origin[1], rnode))